In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import requests
import json

# Configuración
base_url = "http://localhost:8080"
username = "admin"
password = "admin"

# 1. Primero verifica la salud de la API
try:
    health_response = requests.get(f"{base_url}/health")
    print(f"Estado de salud de Airflow: {health_response.status_code}")
    print(health_response.text[:100] if health_response.status_code == 200 else "No disponible")
except Exception as e:
    print(f"Error al verificar salud: {str(e)}")

# 2. Intenta el endpoint de autenticación correcto (funciona con Airflow 2.x y 3.x)
auth_endpoints = [
    "/api/v1/security/login",  # Airflow 3.x
    "/auth/token"              # Airflow 2.x
]

token = None
for endpoint in auth_endpoints:
    try:
        print(f"\nIntentando autenticación con: {endpoint}")
        
        # Formato diferente según el endpoint
        if "security/login" in endpoint:
            auth_data = {"username": username, "password": password}
            auth_response = requests.post(f"{base_url}{endpoint}", json=auth_data)
        else:
            auth_data = {"username": username, "password": password}
            auth_response = requests.post(f"{base_url}{endpoint}", data=auth_data)
        
        print(f"Respuesta: {auth_response.status_code}")
        
        if auth_response.status_code == 200:
            token_data = auth_response.json()
            token = token_data.get("access_token")
            print(f"Token obtenido correctamente: {token[:10]}...")
            break
        else:
            print(f"Error: {auth_response.text}")
    
    except Exception as e:
        print(f"Error con {endpoint}: {str(e)}")

# 3. Si se obtuvo un token, consultar los DAGs
if token:
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {token}"
    }
    
    try:
        response = requests.get(
            f"{base_url}/api/v2/dags?limit=50&offset=0",
            headers=headers
        )
        
        if response.status_code == 200:
            dags_data = response.json()
            print(f"\nTotal de DAGs: {dags_data.get('total_entries', 0)}")
            print(json.dumps(dags_data, indent=2))
        else:
            print(f"\nError al obtener DAGs: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"\nError al consultar DAGs: {str(e)}")
else:
    # 4. Si no se pudo obtener un token, intenta autenticación básica
    print("\nIntentando autenticación básica...")
    session = requests.Session()
    session.auth = (username, password)
    
    try:
        response = session.get(f"{base_url}/api/v2/dags?limit=50&offset=0")
        
        if response.status_code == 200:
            dags_data = response.json()
            print(f"Total de DAGs: {dags_data.get('total_entries', 0)}")
            print(json.dumps(dags_data, indent=2))
        else:
            print(f"Error con autenticación básica: {response.status_code}")
            print(response.text)
    except Exception as e:
        print(f"Error con autenticación básica: {str(e)}")

Estado de salud de Airflow: 404
No disponible

Intentando autenticación con: /api/v1/security/login
Respuesta: 405
Error: {"detail":"Method Not Allowed"}

Intentando autenticación con: /auth/token
Respuesta: 422
Error: {"detail":[{"type":"model_attributes_type","loc":["body"],"msg":"Input should be a valid dictionary or object to extract fields from","input":"username=admin&password=admin"}]}

Intentando autenticación básica...
Error con autenticación básica: 401
{"detail":"Not authenticated"}


In [6]:
import subprocess
import json

# Ejecutar un comando de Airflow CLI para listar los DAGs
result = subprocess.run(
    ["airflow", "dags", "list", "--output", "json"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    dags_data = json.loads(result.stdout)
    print(f"Total de DAGs: {len(dags_data)}")
    print(json.dumps(dags_data, indent=2))
else:
    print(f"Error al obtener DAGs desde CLI: {result.stderr}")

FileNotFoundError: [WinError 2] The system cannot find the file specified